## 1) 대규모 점포 데이터 정리

In [2]:
import pandas as pd
import numpy as np
import pyproj
import folium

In [3]:
df = pd.read_csv('대규모 점포.csv',
                 encoding='cp949',
                 usecols=['좌표정보(X)','좌표정보(Y)'])

### ①중부원점 좌표계 경위도 좌표계로 변환
: 우리가 갖고 있는 '좌표->행정동' 변환 코드는 '경위도 좌표계' 용이기 때문에<br>대규모 점포 데이터의 중부원점 좌표계를 경위도 좌표계로 변환하는 과정을 거칩니다. 

In [4]:
df['좌표정보(X)'] = pd.to_numeric(df['좌표정보(X)'], errors="coerce")
df['좌표정보(Y)'] = pd.to_numeric(df['좌표정보(Y)'], errors="coerce")

df = df.dropna()
df.index=range(len(df))

def project_array(coord, p1_type, p2_type):
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

coord = np.array(df)

p1_type = "epsg:2097"
p2_type = "epsg:4326"

result = project_array(coord, p1_type, p2_type)

C:\Daye\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Daye\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-4-88d3c17a6c9e>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])


In [5]:
df['경도'] = result[:, 0]
df['위도'] = result[:, 1]

df_all= pd.read_csv('대규모 점포.csv',
                 encoding='cp949')

df_merge=pd.merge(df_all,df,on='좌표정보(X)',how='inner')
df_merge

df_merge=df_merge[['관리번호','경도','위도']]
df_merge

,관리번호,경도,위도
0,2014301013007500005,127.004053,37.569279
1,2022314016707500001,126.868429,37.522479
2,2011324013907500014,127.133484,37.538531
3,1998310009607500001,127.069078,37.660387
4,2020316015907500001,126.879949,37.501177
...,...,...,...
1372,2012307018907500009,127.016260,37.608779
1373,2012307018907500011,127.045570,37.622935
1374,2012307018907500012,127.015433,37.603261
1375,2012311013107500008,126.919572,37.606103


### ②경위도 좌표계를 이용하여 행정동별로 수 확인

In [6]:
import pandas as pd
import requests 
import sys
import json
import datetime 



def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

 
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가


def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code # 전처리 함수에서 행정구역코드 리스트 받아서 데이터프레임에 추가
    


APP_KEY = 'dc125ad9de42a4ee620f0bdb2e623811' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

df_merge['행정동명']=''
df_merge['행정동_코드']=''

for i in range(len(df_merge)): 
    x_crd = float(df_merge.loc[i, ['경도']])
    y_crd = float(df_merge.loc[i, ['위도']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    df_merge.loc[i, ['행정동명']] = address
    df_merge.loc[i, ['행정동_코드']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))
    

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
900 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1100 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1300 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [7]:
df_rc=df_merge[['관리번호','행정동_코드']]

rc=df_rc.groupby(df_rc["행정동_코드"],as_index=False).count()

rc.rename(columns={'관리번호':'대규모점포개수'},inplace=True)

for i in range(len(rc)):
    rc['행정동_코드'][i]=rc['행정동_코드'][i][0:8]
    
rc['행정동_코드']=rc['행정동_코드'].astype(int)

rc

<ipython-input-7-737f7b45f831>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc['행정동_코드'][i]=rc['행정동_코드'][i][0:8]


,행정동_코드,대규모점포개수
0,11110530,2
1,11110615,14
2,11110630,4
3,11110650,1
4,11110670,6
...,...,...
301,11740640,1
302,11740650,3
303,11740660,11
304,11740685,2


In [8]:
rc.to_csv("./행정동별_대규모점포개수.csv",encoding='euc-kr',index=False)

## 2) 사업체 데이터 정리(엑셀에서 필요한 부분만 가공하여 사용)

In [7]:
business= pd.read_csv('사업체수.csv',
                 encoding='cp949')
business=business[['행정동','합계']]
business.rename(columns={'행정동':'행정동명'},inplace=True)
business.rename(columns={'합계':'사업체수'},inplace=True)
business

,행정동명,사업체수
0,사직동,"3,574"
1,삼청동,732
2,부암동,599
3,평창동,761
4,무악동,647
...,...,...
443,둔촌1동,17
444,둔촌2동,"1,553"
445,암사1동,"1,918"
446,천호2동,"3,439"


## 3) 일반 음식점 데이터 정리(엑셀에서 필요한 부분만 가공하여 사용)

In [8]:
food= pd.read_csv('음식점.csv',
                 encoding='cp949')
food=food[['동','합계']]
food.rename(columns={'동':'행정동명'},inplace=True)
food.rename(columns={'합계':'음식점수'},inplace=True)
food

,행정동명,음식점수
0,사직동,"1,601"
1,삼청동,397
2,부암동,252
3,평창동,273
4,무악동,87
...,...,...
450,둔촌1동,20
451,둔촌2동,353
452,암사1동,509
453,천호2동,"1,040"


In [ ]:
business.to_csv("./행정동별_사업체수.csv",encoding='euc-kr',index=False)

## 4) 병원 데이터 정리(엑셀에서 필요한 부분만 가공하여 사용)

In [9]:
hospital= pd.read_csv('병원.csv',
                 encoding='cp949')
hospital=hospital[['상호명','행정동명','행정동코드']]
hospital.rename(columns={'행정동코드':'행정동_코드'},inplace=True)
hospital2=hospital[['상호명','행정동_코드']]

hospital3=hospital2.groupby(hospital2["행정동_코드"],as_index=False).count()
hospital3['행정동_코드']=hospital3['행정동_코드'].astype(str)

for i in range(len(hospital3)):
    hospital3['행정동_코드'][i]=hospital3['행정동_코드'][i][0:8]

hospital3['행정동_코드']=hospital3['행정동_코드'].astype(int)
hospital3.rename(columns={'상호명':'병원수'},inplace=True)

<ipython-input-9-98d6207c651b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital3['행정동_코드'][i]=hospital3['행정동_코드'][i][0:8]


In [ ]:
hospital3.to_csv("./행정동별_병원수.csv",encoding='euc-kr',index=False)

## 5) 공원 데이터 정리

In [10]:
park= pd.read_csv('서울시 주요 공원현황.csv',
                 encoding='cp949')

park=park[['공원명','X좌표(WGS84)','Y좌표(WGS84)']]

import pandas as pd
import requests 
import sys
import json
import datetime 



def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

 
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address 

def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code
    


APP_KEY = 'dc125ad9de42a4ee620f0bdb2e623811' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

park['행정동명']=''
park['행정동_코드']=''

for i in range(len(park)): 
    x_crd = float(park.loc[i, ['X좌표(WGS84)']])
    y_crd = float(park.loc[i, ['Y좌표(WGS84)']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    park.loc[i, ['행정동명']] = address
    park.loc[i, ['행정동_코드']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [11]:
park2=park[['공원명','행정동_코드']]

park3=park2.groupby(park2["행정동_코드"],as_index=False).count()
park3['행정동_코드']=park3['행정동_코드'].astype(str)
for i in range(len(park3)):
    park3['행정동_코드'][i]=park3['행정동_코드'][i][0:8]

park3.rename(columns={'공원명':'공원수'},inplace=True)

park3=park3.iloc[:113,:]

park3['행정동_코드']=park3['행정동_코드'].astype(int)

<ipython-input-11-78aea48a8115>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park3['행정동_코드'][i]=park3['행정동_코드'][i][0:8]


In [ ]:
park3.to_csv("./행정동별_공원수.csv",encoding='euc-kr',index=False)

## 6) 카페 데이터 정리

In [19]:
cafe= pd.read_csv('서울카페.csv',
                 encoding='cp949')

cafe1=cafe[['상가업소번호','경도','위도']]

cafe1

,상가업소번호,경도,위도
0,20659878,127.000171,37.583149
1,20676428,127.034746,37.658728
2,28491095,126.842935,37.522055
3,23260840,126.813358,37.580941
4,22791439,127.124193,37.534901
...,...,...,...
17098,18147701,126.928794,37.486030
17099,18149032,126.929163,37.483907
17100,18148766,127.074769,37.639226
17101,18148379,126.930092,37.487015


In [23]:
import pandas as pd
import requests 
import sys
import json
import datetime 



def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

 
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가


def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code # 전처리 함수에서 행정구역코드 리스트 받아서 데이터프레임에 추가
    


APP_KEY = 'dc125ad9de42a4ee620f0bdb2e623811' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

cafe1['행정동명']=''
cafe1['행정동_코드']=''

for i in range(len(cafe1)): 
    x_crd = float(cafe1.loc[i, ['경도']])
    y_crd = float(cafe1.loc[i, ['위도']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    cafe1.loc[i, ['행정동명']] = address
    cafe1.loc[i, ['행정동_코드']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))
    

<ipython-input-23-c00a24ab34a8>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe1['행정동명']=''
<ipython-input-23-c00a24ab34a8>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe1['행정동_코드']=''
C:\Daye\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
900 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1100 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1300 번째 경위도 좌표계에 대한 정보 불러오는 중...
1400 번째 경위도 좌표계에 대한 정보 불러오는 중...
1500 번째 경위도 좌표계에 대한 정보 불러오는 중...
1600 번째 경위도 좌표계에 대한 정보 불러오는 중...
1700 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
1900 번째 경위도 좌표계에 대한 정보 불러오는 중...
2000 번째 경위도 좌표계에 대한 정보 불러오는 중...
2100 번째 경위도 좌표계에 대한 정보 불러오는 중...
2200 번째 경위도 좌표계에 대한 정보 불러오는 중...
2300 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2500 번째 경위도 좌표계에 대한 정보 불러오는 중...
2600 번째 경위도 좌표계에 대한 정보 불러오는 중...
2700 번째 경위도 좌표계에 대한 정보 불러오는 중...
2800 번째 경위도 좌표계에 대한 정보 불러오는 중...
2900 번째 경위도 좌표계에 대한 정보 불러오는 중...
3000 번째 경위도 좌표계에 대한 정보

In [24]:
cafe1

,상가업소번호,경도,위도,행정동명,행정동_코드
0,20659878,127.000171,37.583149,혜화동,1111065000
1,20676428,127.034746,37.658728,쌍문2동,1132067000
2,28491095,126.842935,37.522055,신월4동,1147059000
3,23260840,126.813358,37.580941,방화3동,1150064100
4,22791439,127.124193,37.534901,성내2동,1174065000
...,...,...,...,...,...
17098,18147701,126.928794,37.486030,신림동,1162069500
17099,18149032,126.929163,37.483907,서원동,1162064500
17100,18148766,127.074769,37.639226,하계1동,1135061100
17101,18148379,126.930092,37.487015,신림동,1162069500


In [25]:
cafe1.fillna(0)
cafe2=cafe1[['상가업소번호','행정동_코드']]

cafe3=cafe2.groupby(cafe2["행정동_코드"],as_index=False).count()
cafe3['행정동_코드']=cafe3['행정동_코드'].astype(str)
for i in range(len(cafe3)):
    cafe3['행정동_코드'][i]=cafe3['행정동_코드'][i][0:8]

cafe3.rename(columns={'상가업소번호':'카페수'},inplace=True)
cafe3.fillna(0)
cafe3['행정동_코드']=cafe3['행정동_코드'].astype(int)

<ipython-input-25-074af3ef5e93>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe3['행정동_코드'][i]=cafe3['행정동_코드'][i][0:8]


In [ ]:
cafe3.to_csv("./행정동별_카페수.csv",encoding='euc-kr',index=False)

## 7) 주거단지 데이터 정리(엑셀에서 필요한 부분만 가공하여 사용)

In [20]:
house= pd.read_csv('서울주거단지수.csv',
                 encoding='cp949')

house=house[['동','합계']]
house.rename(columns={'동':'행정동명'},inplace=True)
house.rename(columns={'합계':'주거단지수'},inplace=True)

house

,행정동명,주거단지수
0,합계,"3,015,371"
1,소계,"46,859"
2,사직동,"2,778"
3,삼청동,730
4,부암동,"3,070"
...,...,...
446,둔촌1동,5
447,둔촌2동,"7,962"
448,암사1동,"10,089"
449,천호2동,"9,362"


In [ ]:
house.to_csv("./행정동별_주거단지수.csv",encoding='euc-kr',index=False)